## Softmax classifier approach

This approach assumes that quantifiers are learned as a group and that essentially each q quantifier example is a negative example for quantifiers q' with q not equal to q. The interaction between quantifer 

## Global Imports

In [9]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
from quants import *

### keras imports

In [10]:
# keras imports

import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

import keras

from keras.models import Sequential, Model
from keras.layers import SimpleRNN, LSTM, Embedding, Dense, Conv1D, Conv2D, MaxPooling1D, UpSampling1D, Input, Bidirectional, RepeatVector, TimeDistributed, Dropout, LeakyReLU, Flatten
from keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

TensorFlow version:  2.2.0


In [3]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU'),
 LogicalDevice(name='/device:XLA_GPU:0', device_type='XLA_GPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## Classifier models

In [11]:
natural_quantifiers = [Both(), No(), Every(), Few(), Some(), Most()]

In [12]:
unnatural_quantifiers = [MinMax(2, 10), MinMax(3, 6), Or([MinMax(2, 5), MinMax(10, 20)])]

In [13]:
def DenseClassifier():
    model = Sequential()
    model.add(Embedding(len(symbols), 5, input_length=Quantifier.scenes_len))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='sigmoid'))
    model.add(Dropout(0.1))
    model.add(Dense(len(natural_quantifiers), activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def CNNClassifier():
    model= Sequential()
#     model.add(Embedding(len(symbols), 3, input_length=Quantifier.scenes_len))
    model.add(Conv2D(filters = 1, kernel_size=(1,3), strides=1, activation='sigmoid',padding='valid', input_shape=(Quantifier.scenes_len, 3,1)))
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(len(natural_quantifiers), activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# def RNNClassifier():
#     model = Sequential()
#     model.add(Bidirectional(SimpleRNN(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1))))
#     model.add(Dropout(0.5))
#     model.add(RepeatVector(Quantifier.scenes_len))
#     model.add(SimpleRNN(15, activation='sigmoid', return_sequences=True))
#     model.add(Dropout(0.5))
#     model.add(TimeDistributed(Dense(1)))
#     return model

# def LSTMClassifier():
#     model = Sequential()
#     model.add(LSTM(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1)))
#     model.add(Dropout(0.5))
#     model.add(RepeatVector(Quantifier.scenes_len))
#     model.add(LSTM(15, activation='sigmoid', return_sequences=True))
#     model.add(Dropout(0.5))
#     model.add(TimeDistributed(Dense(1)))
#     return model

def BLSTMClassifier():
    model = Sequential()
    model.add(Embedding(len(symbols), 3, input_length=Quantifier.scenes_len))
    model.add(Bidirectional(LSTM(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1))))
    model.add(Dropout(0.1))
    model.add(RepeatVector(Quantifier.scenes_len))
    model.add(Bidirectional(LSTM(15, activation='sigmoid', return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(TimeDistributed(Dense(len(natural_quantifiers), activation='softmax')))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### learn

In [ ]:
num_epochs = 100
batch_size = 20

def learn(scenes, labels):
    model = DenseClassifier()
#     model = CNNClassifier()
#     model = RNNClassifier()
#     model = LSTMClassifier()
#     model = BLSTMClassifier()
    plot_model(model, to_file='classifier_model.png', show_shapes=True)
    model.fit(scenes, labels, batch_size=batch_size, epochs=num_epochs, verbose=1)
    return model

### test

In [ ]:
generate_random_scenes(scenes_num=100)

In [ ]:
def test_quantifer(model, quantifier):
    test_size = 100

    test_scenes = generate_random_scenes(scenes_num=test_size)
    test_labels = [quantifier.quantify(test_scene) for test_scene in test_scenes]
    test_results = [model.evaluate(test_scene.reshape(1, -1),
                                   verbose=0) for test_scene in test_scenes]
    thresh = np.mean(test_results)
    test_thresh = [test_result < thresh for test_result in test_results]
    print(np.sum(np.array(test_labels) == np.array(test_thresh)))

In [ ]:
with tf.device("/cpu:0"):
# with tf.device("/gpu:0"):
    # generate scenes and labels
    per_q_count = 1000
    scenes = np.vstack([generate_quantified_scenes(q, per_q_count) for q in natural_quantifiers])
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(natural_quantifiers_names)
    encoded_Y = encoder.transform(np.concatenate([[q_name] * per_q_count
                                                  for q_name in natural_quantifiers_names], axis = 0).reshape(-1, 1))
    # convert integers to dummy variables (i.e. one hot encoded)
    labels = np_utils.to_categorical(encoded_Y)

    # fit model
#     model = learn(Quantifier.one_hot(scenes).reshape(6000,100,3,1), labels)
    model = learn(scenes, labels)

In [ ]:
with tf.device("/cpu:0"):
# with tf.device("/gpu:0"):
    test_quantifer(model, natural_quantifiers[0])